In [1]:
import cv2
import numpy as np
def preprocess(img, imgsize, jitter, random_placing=False):
    """
    Image preprocess for yolo input
    Pad the shorter side of the image and resize to (imgsize, imgsize)
    Args:
        img (numpy.ndarray): input image whose shape is :math:`(H, W, C)`.
            Values range from 0 to 255.
        imgsize (int): target image size after pre-processing
        jitter (float): amplitude of jitter for resizing
        random_placing (bool): if True, place the image at random position

    Returns:
        img (numpy.ndarray): input image whose shape is :math:`(C, imgsize, imgsize)`.
            Values range from 0 to 1.
        info_img : tuple of h, w, nh, nw, dx, dy.
            h, w (int): original shape of the image
            nh, nw (int): shape of the resized image without padding
            dx, dy (int): pad size
    """
    h, w, _ = img.shape
    img = img[:, :, ::-1]
    assert img is not None
    #尺寸大小的随机抖动，jitter越大，长宽的的变化越大
    if jitter > 0:
        # add jitter
        dw = jitter * w
        dh = jitter * h
        new_ar = (w + np.random.uniform(low=-dw, high=dw))\
                 / (h + np.random.uniform(low=-dh, high=dh))
    else:
        new_ar = w / h

    if new_ar < 1:
        nh = imgsize
        nw = nh * new_ar
    else:
        nw = imgsize
        nh = nw / new_ar
    nw, nh = int(nw), int(nh)
    #图像填充位置的随机性
    if random_placing:
        dx = int(np.random.uniform(imgsize - nw))
        dy = int(np.random.uniform(imgsize - nh))
    else:
        dx = (imgsize - nw) // 2
        dy = (imgsize - nh) // 2

    img = cv2.resize(img, (nw, nh))
    sized = np.ones((imgsize, imgsize, 3), dtype=np.uint8) * 127
    sized[dy:dy+nh, dx:dx+nw, :] = img

    info_img = (h, w, nh, nw, dx, dy)
    return sized, info_img

In [2]:
jitter=0.1
andom_placing=True
img_size=416

In [21]:
img=cv2.imread('data/1.jpg')
# print(img.shape)
# img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
# img[:,:,2] = img[:,:,2]+10
# img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)
# # cv2.imwrite('img_or.jpg',img)
# cv2.imwrite('img_dis3.jpg',img)

In [4]:
sized, info_img=preprocess(img, img_size, jitter=jitter,random_placing=andom_placing)
# print(sized.shape)
# sized=sized[:,:,::-1]
# cv2.imshow('imgs',img)
# cv2.imshow('img',sized)
cv2.waitKey()

-1

In [5]:
def random_distort(img, hue, saturation, exposure):
    """
    perform random distortion in the HSV color space.
    Args:
        img (numpy.ndarray): input image whose shape is :math:`(H, W, C)`.
            Values range from 0 to 255.
        hue (float): random distortion parameter.
        saturation (float): random distortion parameter.
        exposure (float): random distortion parameter.
    Returns:
        img (numpy.ndarray)
    """
    #hue 调整色彩度，sat 调整对比度， exp调整亮度
    dhue = np.random.uniform(low=-hue, high=hue)
    dsat = rand_scale(saturation)
    dexp = rand_scale(exposure)

    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    img = np.asarray(img, dtype=np.float32) / 255.
    img[:, :, 1] *= dsat
    img[:, :, 2] *= dexp
    H = img[:, :, 0] + dhue

    if dhue > 0:
        H[H > 1.0] -= 1.0
    else:
        H[H < 0.0] += 1.0

    img[:, :, 0] = H
    img = (img * 255).clip(0, 255).astype(np.uint8)
    img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)
    img = np.asarray(img, dtype=np.float32)

    return img

In [6]:
def rand_scale(s):
    """
    calculate 
    random scaling factor
    Args:
        s (float): range of the random scale.
    Returns:
        random scaling factor (float) whose range is
        from 1 / s to s .
    """
    scale = np.random.uniform(low=1, high=s)
    if np.random.rand() > 0.5:
        return scale
    return 1 / scale

In [25]:
hue=0.2
saturation=1.5
exposure=1.5

In [26]:
img2 = random_distort(sized, hue, saturation, exposure)
sized=sized[:,:,::-1]
# img2 = cv2.cvtColor(img2, cv2.COLOR_HSV2RGB)
img2=img2[:,:,::-1]
cv2.imwrite('img_or.jpg',img)
cv2.imwrite('img_dis.jpg',img2)
# cv2.waitKey()

True